In [1]:
from util import load_data
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os
from load_clean_data import load_clean_data

/Users/yixing/anaconda3/envs/pyspark_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
spark = SparkSession.builder \
    .appName("Airline Twitter Sentiment Analysis") \
    .getOrCreate()

df = load_clean_data(data_dir='Twitter_Airline Dataset', spark_session=spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/04 02:30:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Show first 5 rows
df.show(5)

+---------+--------------------+-------+----------+-------+--------------------+--------+--------------+------+----------+--------+-------------+-----------------+--------------+----------+----------------------+-----------+-------------------+-------------+--------------------+-------------+-------------+--------------------+---------------+---------------+
| _unit_id|         _created_at|_golden|       _id|_missed|         _started_at|_tainted|      _channel|_trust|_worker_id|_country|          _ip|airline_sentiment|negativereason|   airline|airline_sentiment_gold|       name|negativereason_gold|retweet_count|                text|tweet_created|     tweet_id|       user_timezone|negativereason1|negativereason2|
+---------+--------------------+-------+----------+-------+--------------------+--------+--------------+------+----------+--------+-------------+-----------------+--------------+----------+----------------------+-----------+-------------------+-------------+--------------------

In [6]:
df.printSchema()

root
 |-- _unit_id: string (nullable = true)
 |-- _created_at: string (nullable = true)
 |-- _golden: string (nullable = true)
 |-- _id: integer (nullable = true)
 |-- _missed: string (nullable = true)
 |-- _started_at: string (nullable = true)
 |-- _tainted: string (nullable = true)
 |-- _channel: string (nullable = true)
 |-- _trust: double (nullable = true)
 |-- _worker_id: integer (nullable = true)
 |-- _country: string (nullable = true)
 |-- _ip: string (nullable = true)
 |-- airline_sentiment: string (nullable = true)
 |-- negativereason: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- airline_sentiment_gold: string (nullable = true)
 |-- name: string (nullable = true)
 |-- negativereason_gold: string (nullable = true)
 |-- retweet_count: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- tweet_created: string (nullable = true)
 |-- tweet_id: double (nullable = true)
 |-- user_timezone: string (nullable = true)
 |-- negativereason1: string (nu

# Sentiment Analysis Test

In [3]:
from get_sentiment import SentimentAnalysis
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('sentiwordnet')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/yixing/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/yixing/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/yixing/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/yixing/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
sentiment_class = SentimentAnalysis()

In [5]:
from pyspark.sql.functions import udf, pandas_udf
from pyspark.sql.types import FloatType, StringType
import pandas as pd


In [6]:
os.environ['OBJC_DISABLE_INITIALIZE_FORK_SAFETY'] = 'YES'


In [7]:
# Extract the text, id and airline_sentiment columns
extracted_df = df.select("text", "_id", "airline_sentiment_gold").toPandas()

extracted_df


,text,_id,airline_sentiment_gold
0,*On the brink of bankruptcy. ?�?@JetBlue: Our ...,1.574949e+09,None
1,",@USAirways 2nd time this occurred in 3 weeks....",1.574991e+09,None
2,. @SouthwestAir condescension must be a qualit...,1.574837e+09,None
3,. @united sad that you don't think it is impor...,1.574908e+09,None
4,. @united you know your updates every 6 hours ...,1.575289e+09,None
...,...,...,...
61323,“@SouthwestAir: @saysorrychris Can you follow ...,1.575269e+09,None
61324,“@united: @BattierCCIpuppy We hope to see you ...,1.574999e+09,None
61325,“@united: @BattierCCIpuppy We hope to see you ...,1.575030e+09,None
61326,“@united: @mllovelace The baggage fee is waive...,1.574917e+09,None


In [8]:
# Apply the sentiment analysis function to the text column
extracted_df['sentiment'] = extracted_df['text'].apply(lambda x: sentiment_class.get_sentiment(x))

In [26]:
extracted_df = extracted_df[(extracted_df['airline_sentiment_gold'].notnull()) & (extracted_df['text'].notnull()) & (extracted_df['sentiment'].notnull())]

In [28]:
# Check how accurate the sentiment analysis is based on the airline_sentiment_gold column output is 0 - 1
# Calculate matches by comparing the 'airline_sentiment_gold' and 'sentiment' columns
matches = (extracted_df['airline_sentiment_gold'] == extracted_df['sentiment'])

# Calculate accuracy as the number of matches divided by the total number of rows
accuracy = matches.sum() / len(matches)

accuracy



0.3615490737091052

In [29]:
count = 0
total = len(extracted_df)

print(f"Total: {total}")
# Loop through the extracted_df
for i in range(len(extracted_df)):
    # If the airline_sentiment_gold column is not null
    if extracted_df['airline_sentiment_gold'][i] is not None:
        # If the sentiment column is not equal to the airline_sentiment_gold column
        if extracted_df['sentiment'][i] != extracted_df['airline_sentiment_gold'][i]:
            # Print the text, sentiment and airline_sentiment_gold columns
            count += 1

accuracy = 1 - count / total
print(f"Count: {count}")
print(f"Total: {total}")
print(f"Accuracy: {accuracy}")

KeyError: 0

In [10]:
def test_function(string):
    return f"{string} + test"

test_function_udf = udf(test_function, StringType())

df = df.withColumn('test', test_function_udf(col('text')))

df.show(5)

24/04/04 02:19:09 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+--------------------+-------+----------+-------+--------------------+--------+--------------+------+----------+--------+-------------+-----------------+--------------+----------+----------------------+-----------+-------------------+-------------+--------------------+-------------+-------------+--------------------+---------------+---------------+--------------------+
| _unit_id|         _created_at|_golden|       _id|_missed|         _started_at|_tainted|      _channel|_trust|_worker_id|_country|          _ip|airline_sentiment|negativereason|   airline|airline_sentiment_gold|       name|negativereason_gold|retweet_count|                text|tweet_created|     tweet_id|       user_timezone|negativereason1|negativereason2|                test|
+---------+--------------------+-------+----------+-------+--------------------+--------+--------------+------+----------+--------+-------------+-----------------+--------------+----------+----------------------+-----------+------------

In [13]:
# Show first 5 rows
df.show(5)

objc[51286]: +[__NSCFConstantString initialize] may have been in progress in another thread when fork() was called.
objc[51286]: +[__NSCFConstantString initialize] may have been in progress in another thread when fork() was called. We cannot safely call it or ignore it in the fork() child process. Crashing instead. Set a breakpoint on objc_initializeAfterForkError to debug.
24/04/04 01:50:53 ERROR Executor: Exception in task 0.0 in stage 4.0 (TID 8)
org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:601)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:583)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:99)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.re

Py4JJavaError: An error occurred while calling o53.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 8) (14.8.126.129.static.m1net.com.sg executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:601)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:583)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:99)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:514)
	at org.apache.spark.InterruptibleIterator.hasNext(In